In [ ]:
import pickle 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn

from datetime import datetime

In [ ]:
scaler_minmax = MinMaxScaler()

In [ ]:
def get_data(df, time_step):

    X, Y, Date = [], [], []

    for i in range(len(df) - time_step - 1):

        x = df[i:(i+time_step)]['Price']
        X.append(x)
        Y.append(df.iloc[i+time_step]['Price'])
        Date.append(df.iloc[i+time_step]['Date'].to_pydatetime().strftime('%Y-%m-%d'))

    return np.array(X), np.array(Y), np.array(Date)

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.shape[0], self.hidden_size)
        c0 = torch.zeros(self.num_layers, x.shape[0], self.hidden_size)
        
        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = self.fc(out[:,-1,:])
        return out

In [ ]:
def train(X, y, model, lr, cost_fn, epochs):
    
    # Define the optimizer    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):

        # Training
        model.train()
            
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(X)

        # Compute the loss
        loss = cost_fn(outputs, y)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        if (epoch+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
    
    return model

In [ ]:
## Ref: https://medium.com/codex/how-to-calculate-bollinger-bands-of-a-stock-with-python-f9f7d1184fc3

def get_sma(prices, rate):
    return prices.rolling(rate).mean()

def get_bollinger_bands(prices, rate=20):
    sma = get_sma(prices, rate)
    std = prices.rolling(rate).std()
    bollinger_up = sma + std * 2 # Calculate top band
    bollinger_down = sma - std * 2 # Calculate bottom band
    return sma, bollinger_up, bollinger_down

In [ ]:
def get_weekly_data(df, time_step):

    X, Y, Date = [], [], []

    for i in range(time_step, len(df), 5):

        x = df[i-time_step : i]['Price']
        X.append(x)
        Y.append(df[i:i+5]['Price'])
        Date.append(df[i:i+5]['Date'].dt.strftime('%Y-%m-%d'))

    return np.array(X), np.array(Y), np.array(Date)

In [ ]:
def get_monthly_data(df, time_step, maxdays):

    X, Y, Date = [], [], []

    for i in range(time_step, len(df), maxdays):

        x = df[i-time_step : i]['Price']
        X.append(x)
        Y.append(df[i:i+maxdays]['Price'])
        Date.append(df[i:i+maxdays]['Date'].dt.strftime('%Y-%m-%d'))

    return np.array(X), np.array(Y), np.array(Date)

In [ ]:
def numpy2tensor(x,y, daily=True):
  x = torch.from_numpy(x).type(torch.Tensor).view(x.shape[0], x.shape[1], 1)
  if daily==True:
    y = torch.from_numpy(y).type(torch.Tensor).view(y.shape[0], 1)
  else:
    y = torch.from_numpy(y).type(torch.Tensor)
  return x,y

In [ ]:
def scale_back(x, max, min):
  return (x * (max - min)) + min